## Build a DELWAQ D-Emissions model from scratch

This notebook demonstrates how to prepare **DELWAQ D-Emissions** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

In hydroMT, you can interact with **DELWAQ** models either to be used by **D-Emissions (EM)** or **D-Water Quality (WQ)**.

Lets first check if the delwaq model is recognized by hydromt

In [1]:
# this should return "delwaq, wflow, wflow_sediment"
!hydromt --models

hydroMT model plugins: wflow, wflow_sediment, delwaq


Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT REGION

  Build models from source data.

  Example usage:
  --------------

  To build a wflow model for a subbasin using and point coordinates snapped to cells with stream order >= 4
  hydromt build wflow /path/to/model_root "{'subbasin': [-7.24, 62.09], 'strord': 4}" -i /path/to/wflow_config.ini

  To build a wflow model based on basin ID
  hydromt build wflow /path/to/model_root "{'basin': 230001006}"

  To build a sfincs model based on a bbox (for Texel)
  hydromt build sfincs /path/to/model_root "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

Options:
  -r, --res FLOAT             Model resolution in model src.
  --build-base / --build-all  Deprecated!
  --opt TEXT                  Component specific keyword arguments, see the
                              setup_<component> method of the specific model
                              for more information about the arguments.

  -i, --config PATH           Path to hydroMT configuration fi

### Setup delwaq EM model base layers 

DELWAQ models are quite specific in the sense that they are usually added on top of an already existing hydrologic / hydraulic or hydrodynamic model.
In hydroMT, for now, DELWAQ models (EM for D-Emissions and WQ for D-Water Quality) can only be built on top of [Wflow](https://deltares.github.io/Wflow.jl/stable/) hydrologic models. You can find more information on this coupling in [docs(coupling_wflow)](https://deltares.github.io/hydromt_delwaq/latest/advanced/coupling_wflow.html).

Here is how you can build a DELWAQ model:

In [3]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build delwaq "./EM_test_base" "{'wflow': 'wflow_piave'}" --opt global.mtype=EM -vvv

2021-05-06 15:29:09,374 - build - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_base/hydromt.log.
2021-05-06 15:29:09,374 - build - log - INFO - HydroMT version: 0.4.0
2021-05-06 15:29:09,374 - build - main - INFO - Building instance of delwaq model at /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_base.
2021-05-06 15:29:09,374 - build - main - INFO - User settings:
2021-05-06 15:29:09,375 - build - cli_utils - INFO - global.mtype: EM
2021-05-06 15:29:09,375 - build - model_api - DEBUG - setup_basemaps.region: {'wflow': 'wflow_piave'}
2021-05-06 15:29:09,375 - build - model_api - DEBUG - setup_basemaps.include_soil: False
2021-05-06 15:29:09,375 - build - basin_mask - DEBUG - Parsed region (kind=model): {'mod': <hydromt_wflow.wflow.WflowModel object at 0x7f84c0e53880>}
2021-05-06 15:29:09,376 - build - delwaq - INFO - Preparing EM basemaps from hydromodel.
2021-05-06 15:29:09

2021-05-06 15:29:09,465 - build - delwaq - INFO - Write model data to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_base
2021-05-06 15:29:09,471 - build - delwaq - INFO - Writing staticmap files.
2021-05-06 15:29:09,478 - build - delwaq - INFO - Writing model config to file.
2021-05-06 15:29:09,479 - build - delwaq - INFO - Writing hydromap files.


2021-05-06 15:29:09,523 - build - delwaq - INFO - Writting geometry file in root/staticdata
2021-05-06 15:29:09,529 - build - delwaq - WARNING - Warning: no dynamic map, skipping write_dynamicmaps.


The example above means the following: run **hydromt build** with:

* `delwaq` : i.e. build a delwaq model
* `./EM_test_base` : output model folder
* `"{'wflow': 'wflow_piave'}"` : derive the model from the existing wflow model located in wflow_piave folder. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options). For delwaq, the only possible options is from an existing wflow model.
* `-opt global.mtype=EM` : specifies which type of delwaq model to build. Here **EM** for D-Emissions.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (grid, segment IDs) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delwaq/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html). 

**Almost each DELWAQ model is unique** depending on which substances and sources are included but also what kind of emission data is available. For these reasons, there is no default build ini file for a delwaq model and still some manual steps required to build and run a D-Emissions model with hydroMT. You can learn more about these steps in the [docs(hydromt_D-Emissions)](https://deltares.github.io/hydromt_delwaq/latest/advanced/generic_delwaq.html#emissions-modelling-with-d-emission).

We will load an example delwaq build EM ini file for inspection:

In [4]:
fn_ini = 'delwaq_build_EM.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
mtype             = EM                    # type of Delwaq model ['EM', 'WQ']
data_libs         = []                    # add optional paths to data yml files

[setup_basemaps]
include_soil      = False                 # add a soil compartment in addition to the surface waters

[setup_monitoring]
mon_points        = None                  # monitoring points ['None', 'segments', 'path to station location', 'source in DataCatalog']
mon_areas         = compartments          # monitoring areas ['None', 'compartments', 'subcatch']

[setup_hydrology_forcing]
hydro_forcing_fn  = None                  # source name of the hydrological forcing in the yaml file
starttime         = 2010-02-03 00:00:00   # start time of the Delwaq run
endtime           = 2010-02-10 00:00:00   # end time of the Delwaq run
timestepsecs      = 86400                 # model timestep in seconds
add_volume_offset = True                  # add a one-timestep offset to the volume data in the hydrological forcing 

Some explanations for the sections the ini file above:

* **global**: options true for every model components of delwaq. This is where you can specify if you want to build an **EM** or **WQ** model using the *mtype* option. You can also add local data libaries in *data_libs* instead of in the command line.
* **setup_basemaps**: prepares the first delwaq model layers (grid, segment ID) and for EM also the geometry.
* **setup_monitoring**: prepares no monitoring points and monitoring areas by compartments (emissions only here).
* **setup_hydrology_forcing**: prepares hydrological data from wflow outputs. Here we skip this step by putting *hydro_forcing_fn=None*.
* **setup_emission_raster**: prepares emissions grid from a raster file. You can repeat this section several times for several rasters. Here we will prepare population and urban/rural classification data from the global *GHS-POP_2015* and *GHS-SMOD_2015* data.
* **setup_emission_vector**: prepares emissions grid from a vector file. You can repeat this section several times for several rasters. Here we will prepare GDP data from the global *GDP_world* data.
* **setup_emission_mapping**: prepares emissions grid from a region vector file (exemple administrative levels) and an associated table with statistics. Here we skip this type of data preparation. 

For more information on all the options used (resampling method, NaN values handling...), please check the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html)

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which are available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `--dd` flag. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete delwaq EM model 

In [5]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build delwaq "./EM_test_full" "{'wflow': 'wflow_piave'}" -i delwaq_build_EM.ini -vv

2021-05-06 15:29:11,833 - build - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full/hydromt.log.
2021-05-06 15:29:11,833 - build - log - INFO - HydroMT version: 0.4.0
2021-05-06 15:29:11,833 - build - main - INFO - Building instance of delwaq model at /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full.
2021-05-06 15:29:11,833 - build - main - INFO - User settings:
2021-05-06 15:29:11,834 - build - cli_utils - INFO - global.mtype: EM
2021-05-06 15:29:11,835 - build - cli_utils - INFO - global.data_libs: []
2021-05-06 15:29:11,835 - build - cli_utils - INFO - setup_basemaps.include_soil: False
2021-05-06 15:29:11,835 - build - cli_utils - INFO - setup_monitoring.mon_points: None
2021-05-06 15:29:11,835 - build - cli_utils - INFO - setup_monitoring.mon_areas: compartments
2021-05-06 15:29:11,835 - build - cli_utils - INFO - setup_hydrology_forcing.hydro_forcing_fn: None
2021-0

2021-05-06 15:29:11,928 - build - model_api - DEBUG - setup_monitoring.mon_points: None
2021-05-06 15:29:11,928 - build - model_api - DEBUG - setup_monitoring.mon_areas: compartments
2021-05-06 15:29:11,928 - build - delwaq - INFO - Setting monitoring points and areas
2021-05-06 15:29:11,928 - build - delwaq - INFO - No monitoring points set in the config file, skipping
2021-05-06 15:29:11,956 - build - model_api - DEBUG - setup_hydrology_forcing.hydro_forcing_fn: None
2021-05-06 15:29:11,956 - build - model_api - DEBUG - setup_hydrology_forcing.starttime: 2010-02-03 00:00:00
2021-05-06 15:29:11,956 - build - model_api - DEBUG - setup_hydrology_forcing.endtime: 2010-02-10 00:00:00
2021-05-06 15:29:11,956 - build - model_api - DEBUG - setup_hydrology_forcing.timestepsecs: 86400
2021-05-06 15:29:11,956 - build - model_api - DEBUG - setup_hydrology_forcing.add_volume_offset: True
2021-05-06 15:29:11,956 - build - data_adapter - INFO - Updating data sources from yml file /home/runner/.hydr

2021-05-06 15:29:12,038 - build - delwaq - WARNING - None or Invalid source 'None', skipping setup_dynamic.
2021-05-06 15:29:12,038 - build - model_api - DEBUG - setup_emission_raster.emission_fn: GHS-POP_2015
2021-05-06 15:29:12,038 - build - model_api - DEBUG - setup_emission_raster.scale_method: average
2021-05-06 15:29:12,038 - build - model_api - DEBUG - setup_emission_raster.fillna_method: zero
2021-05-06 15:29:12,038 - build - model_api - DEBUG - setup_emission_raster.fillna_value: 0.0
2021-05-06 15:29:12,038 - build - model_api - DEBUG - setup_emission_raster.area_division: True
2021-05-06 15:29:12,038 - build - delwaq - INFO - Preparing 'GHS-POP_2015' map.
2021-05-06 15:29:12,049 - build - data_adapter - INFO - DataCatalog: Getting GHS-POP_2015 RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.2/GHS-POP_2015.tif


2021-05-06 15:29:12,078 - build - data_adapter - DEBUG - RasterDataset: Clip with geom - [11.775, 45.808, 12.742, 46.692]
2021-05-06 15:29:12,118 - build - emissions - INFO - Deriving None using average resampling (nodata=-200.0).


2021-05-06 15:29:12,153 - build - model_api - DEBUG - setup_emission_raster.emission_fn: GHS-SMOD_2015
2021-05-06 15:29:12,153 - build - model_api - DEBUG - setup_emission_raster.scale_method: nearest
2021-05-06 15:29:12,153 - build - model_api - DEBUG - setup_emission_raster.fillna_method: zero
2021-05-06 15:29:12,154 - build - model_api - DEBUG - setup_emission_raster.fillna_value: 0.0
2021-05-06 15:29:12,154 - build - model_api - DEBUG - setup_emission_raster.area_division: False
2021-05-06 15:29:12,154 - build - delwaq - INFO - Preparing 'GHS-SMOD_2015' map.
2021-05-06 15:29:12,164 - build - data_adapter - INFO - DataCatalog: Getting GHS-SMOD_2015 RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.2/GHS-SMOD_2015.tif


...

2021-05-06 15:29:12,539 - build - model_api - DEBUG - setup_emission_mapping.source_name: None
2021-05-06 15:29:12,539 - build - delwaq - WARNING - Source name set to None, skipping setup_emission_mapping.
2021-05-06 15:29:12,539 - build - delwaq - INFO - Write model data to /home/runner/work/hydromt_delwaq/hydromt_delwaq/docs/examples/examples/EM_test_full


2021-05-06 15:29:12,556 - build - delwaq - INFO - Writing staticmap files.


2021-05-06 15:29:12,675 - build - delwaq - INFO - Writing model staticgeom to file.


2021-05-06 15:29:12,697 - build - delwaq - INFO - Writing model config to file.
2021-05-06 15:29:12,697 - build - delwaq - INFO - Writing hydromap files.
2021-05-06 15:29:12,740 - build - delwaq - INFO - Writting geometry file in root/staticdata


2021-05-06 15:29:12,749 - build - delwaq - WARNING - Warning: no dynamic map, skipping write_dynamicmaps.


With this example we build a complete delwaq EM model including emission data. Compared to the previous **hydromt build** we have added:
* `-i delwaq_build_EM.ini` : setup configuration file including all components to build and their arguments

Note that the `--opt global.mtype=EM` is not present anymore, as this option is directly set in the delwaq_build_EM.ini file.

Next we check which files and folders have been created:

* **config**: basic configuration setups for the Delwaq input file .inp. Names of the ASCII files start with the corresponding block in the .inp file.
* **dynamicdata**: Delwaq time-dependant data (hydrological fluxes) in binary format.
* **fews**: additional files for connection with Delft-FEWS (not implemented yet)
* **hydromodel**: data extracted to connect the hydrological model (wflow) and Delwaq (tif format). These files are not mandatory for DELWAQ but produced extra from HydroMT.
* **staticdata**: static (spatial but non time-dependant) data for Delwaq (typically emission data) in binary format. A copy of the data in NetCDF format is also available for easier visualization.
* **staticgeoms**: related geometry files (geojson format). These files are not mandatory for DELWAQ but produced extra from HydroMT.

In [6]:
import os
root = 'EM_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

EM_test_full
 - hydromt.log
EM_test_full/fews
EM_test_full/dynamicdata
EM_test_full/staticgeoms
 - monareas.geojson
 - basins.geojson
EM_test_full/staticdata
 - GHS-POP_2015.dat
 - staticmaps.nc
 - GHS-SMOD_2015.dat
 - slope.dat
 - streamorder.dat
 - GDP_world.dat
EM_test_full/hydromodel
 - rivwth.tif
 - ldd.tif
 - ptid.tif
 - basins.tif
 - modelmap.tif
 - rivlen.tif
EM_test_full/config
 - B4_nrofexch.inc
 - B2_nrofmon.inc
 - B2_stations.inc
 - B5_boundlist.inc
 - B7_geometry-parameters.inc
 - B2_monareas.inc
 - B7_geometry.inc
 - B3_attributes.inc
 - B7_geometry.bin
 - B3_nrofseg.inc


You can see that the dynamicdata folder (hydrological input) does not contain any file yet. We will add them in the **update forcing** example notebook.

### Visualize and/or inspect model schematization

* The **delwaq plot** example notebook contains scripts to visualize your model (staticdata, dynamicdata, hydromodel and staticgeoms).

In the meantime, feel free to have a look at some configuration files that were prepared by HydroMT. You can change the file name from the example code below.

In [7]:
import os
model_path = './EM_test_full'
fn_config = 'config/B3_nrofseg.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

1653 ; nr of segments



In the config folder, you can also see three geometry files:

* B7_geometry-parameters.inc (Headers for the binary version)
* B7_geometry.bin (Binary version)
* B7_geometry.inc (ASCII version)

These files are specific for the D-Emissions plugin and specifies geometry properties of the D-Emissions segments.
These properties are:

* Segment area (m2)
* Fraction of the segment area that is paved
* Fraction of the segment area that is unpaved
* Fraction of the segment area that is open water

In [8]:
import os
model_path = './EM_test_full'
fn_config = 'config/B7_geometry-parameters.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

PARAMETERS TotArea fPaved fUnpaved fOpenWater

